In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')
import kaleido

In [2]:
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [3]:

def ode_model(z, t, beta, sigma, gamma, mu):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """
    S, E, I, R, D = z
    N = S + E + I + R + D
    dSdt = -beta*S*I/N
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I - mu*I
    dRdt = gamma*I
    dDdt = mu*I
    return [dSdt, dEdt, dIdt, dRdt, dDdt]


def ode_solver(t, initial_conditions, params):
    initE, initI, initR, initN, initD = initial_conditions
    beta, sigma, gamma, mu = params
    initS = initN - (initE + initI + initR + initD)
    res = odeint(ode_model, [initS, initE, initI, initR, initD], t, args=(beta, sigma, gamma, mu))
    return res

# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 32000000 
# S0 = 966000000
initE = 1
initI = 1
initR = 0
initD = 0
sigma = 1/5.2
gamma = 1/2.9
mu = 0.034
R0 = 4
beta = R0 * gamma
days = 180
vacunados=0 

def main(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days, vacunados):
    initial_conditions = [initE, initI, initR, initN-vacunados, initD]
    params = [beta, sigma, gamma, mu]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, I, R, D = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Expuesta'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infectada'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recuperada'))
    fig.add_trace(go.Scatter(x=tspan, y=D, mode='lines+markers',name='Muerta'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulation of SEIRD Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.show()

In [4]:

interact(main, 
         initE=IntSlider(min=0, max=100000, step=1, value=initE, description='Pobl. Exp.', style=style, layout=slider_layout),
         initI=IntSlider(min=0, max=100000, step=10, value=initI, description='Pobl. Infect.', style=style, layout=slider_layout),
         initR=IntSlider(min=0, max=100000, step=10, value=initR, description='Pobl. Recup.', style=style, layout=slider_layout),
         initD=IntSlider(min=0, max=100000, step=10, value=initD, description='Pobl. Muerta', style=style, layout=slider_layout),
         initN=IntSlider(min=0, max=32000000, step=1000, value=initN, description='Población Sucep.', style=style, layout=slider_layout),
         beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Ratio de Inf.', style=style, layout=slider_layout),
         sigma=FloatSlider(min=0, max=4, step=0.01, value=sigma, description='Ratio de Incub.', style=style, layout=slider_layout),
         gamma=FloatSlider(min=0, max=4, step=0.01, value=gamma, description='Ratio de Recup.', style=style, layout=slider_layout),
         mu=FloatSlider(min=0, max=1, step=0.01, value=mu, description='Ratio de Mort.', style=style, layout=slider_layout),
         days=IntSlider(min=1, max=600, step=7, value=days, description='Días', style=style, layout=slider_layout),
         vacunados=IntSlider(min=1, max=32000000, step=1000, value=vacunados, description='Vacunados', style=style, layout=slider_layout)
        )

interactive(children=(IntSlider(value=1, description='Pobl. Exp.', layout=Layout(width='99%'), max=100000, sty…

<function __main__.main(initE, initI, initR, initD, initN, beta, sigma, gamma, mu, days, vacunados)>